In [2]:
import os as os
os.environ['TF_CPP_MIN_LOG_LEVEL']='0'

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import SparseCategoricalCrossentropy

2023-02-27 16:23:36.357802: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 16:23:37.160026: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-27 16:23:37.160094: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-27 16:23:40.864901: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
data=pd.read_csv('./data/train.csv')

In [3]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
y_=data.pop('label')

In [5]:
y=np.array([y_]).reshape(42000,-1)
X=np.array(data)

In [8]:
X_train,X_,y_train,y_=train_test_split(X,y,test_size=0.4,random_state=1)
X_cv,X_test,y_cv,y_test=train_test_split(X_,y_,test_size=0.5,random_state=1)
print(X_train.shape,y_train.shape)
print(X_cv.shape,y_cv.shape)
print(X_test.shape,y_test.shape)


(25200, 784) (25200, 1)
(8400, 784) (8400, 1)
(8400, 784) (8400, 1)


### Without using from_logits=True

In [10]:
model=tf.keras.Sequential([
    Dense(100,activation='relu',name='l1'),
    Dense(60,activation='relu',name='l2'),
    Dense(80,activation='relu',name='l3'),
    Dense(40,activation='linear',name='l4'),
    Dense(10,activation='softmax')
])

In [11]:
model.compile(loss=SparseCategoricalCrossentropy())
model.fit(X_train,y_train,epochs=100)

2023-02-27 16:25:16.001031: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 158054400 exceeds 10% of free system memory.


Epoch 1/100
788/788 [==============================] - 5s 4ms/step - loss: 2.0175
Epoch 2/100
788/788 [==============================] - 3s 4ms/step - loss: 0.3729
Epoch 3/100
788/788 [==============================] - 3s 4ms/step - loss: 0.2667
Epoch 4/100
788/788 [==============================] - 3s 4ms/step - loss: 0.2224
Epoch 5/100
788/788 [==============================] - 3s 4ms/step - loss: 0.2019
Epoch 6/100
788/788 [==============================] - 3s 4ms/step - loss: 0.1882
Epoch 7/100
788/788 [==============================] - 3s 4ms/step - loss: 0.1744
Epoch 8/100
788/788 [==============================] - 3s 4ms/step - loss: 0.1614
Epoch 9/100
788/788 [==============================] - 3s 4ms/step - loss: 0.1550
Epoch 10/100
788/788 [==============================] - 3s 4ms/step - loss: 0.1518
Epoch 11/100
788/788 [==============================] - 3s 4ms/step - loss: 0.1324
Epoch 12/100
788/788 [==============================] - 3s 4ms/step - loss: 0.1344
Epoch 13/100


In [38]:
y_cv_hat_prediction=model(X_cv)
y_train_hat_prediction=model(X_train)
print(y_cv_hat_prediction.shape)
print(y_train_hat_prediction.shape)


(8400, 10)
(25200, 10)


In [44]:
def predicted_value(x):
    m=x.shape[0]
    y=np.zeros((m,1),dtype=int)
    for i in range(m):
        y[i][0]=np.argmax(x[i])
    return y


In [47]:
y_cv_hat=np.array(predicted_value(y_cv_hat_prediction))
y_train_hat=np.array(predicted_value(y_train_hat_prediction))
print(y_cv_hat.shape)
print(y_train_hat.shape)


(8400, 1)
(25200, 1)


In [66]:
def check_accuracy_classification(y,y_hat):
    m=y.shape[0]
    misclassification=0
    for i in range(m):
        if(y[i][0]!=y_hat[i][0]):
            misclassification+=1
    
    return misclassification/m

In [49]:
print(f"Training Error: {check_accuracy_classification(y_train,y_train_hat)*100}%")
print(f"Dev Error: {check_accuracy_classification(y_cv,y_cv_hat)*100}%")


Training Error: 0.16666666666666669%
Dev Error: 3.166666666666667%


In [55]:
preferred_model=tf.keras.Sequential([
    tf.keras.layers.InputLayer((784,)),
    Dense(100,activation='relu',name='l1'),
    Dense(60,activation='relu',name='l2'),
    Dense(80,activation='relu',name='l3'),
    Dense(40,activation='linear',name='l4'),
    Dense(10,activation='linear')
])

In [56]:
preferred_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 l1 (Dense)                  (None, 100)               78500     
                                                                 
 l2 (Dense)                  (None, 60)                6060      
                                                                 
 l3 (Dense)                  (None, 80)                4880      
                                                                 
 l4 (Dense)                  (None, 40)                3240      
                                                                 
 dense_2 (Dense)             (None, 10)                410       
                                                                 
Total params: 93,090
Trainable params: 93,090
Non-trainable params: 0
_________________________________________________________________


In [57]:
preferred_model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
   
)
preferred_model.fit(X_train,y_train,epochs=100)

Epoch 1/100
788/788 [==============================] - 5s 5ms/step - loss: 1.8213
Epoch 2/100
788/788 [==============================] - 4s 5ms/step - loss: 0.3597
Epoch 3/100
788/788 [==============================] - 4s 4ms/step - loss: 0.2597
Epoch 4/100
788/788 [==============================] - 4s 4ms/step - loss: 0.2202
Epoch 5/100
788/788 [==============================] - 4s 4ms/step - loss: 0.1938
Epoch 6/100
788/788 [==============================] - 3s 4ms/step - loss: 0.1636
Epoch 7/100
788/788 [==============================] - 4s 5ms/step - loss: 0.1513
Epoch 8/100
788/788 [==============================] - 4s 5ms/step - loss: 0.1422
Epoch 9/100
788/788 [==============================] - 4s 5ms/step - loss: 0.1273
Epoch 10/100
788/788 [==============================] - 4s 5ms/step - loss: 0.1268
Epoch 11/100
788/788 [==============================] - 4s 5ms/step - loss: 0.1177
Epoch 12/100
788/788 [==============================] - 4s 4ms/step - loss: 0.1045
Epoch 13/100


In [58]:
y_cv_hat_pred_preferred=preferred_model(X_cv)
y_train_hat_pred_preferred=preferred_model(X_train)

In [60]:

y_cv_hat_pred=predicted_value(y_cv_hat_pred_preferred)
y_train_hat_pred=predicted_value(y_train_hat_pred_preferred)

In [67]:
print(f"Training Error Preferred Model: {check_accuracy_classification(y_train,y_train_hat_pred)*100}%")
print(f"Dev Error Preferred Model: {check_accuracy_classification(y_cv,y_cv_hat_pred)*100}%")


Training Error Preferred Model: 0.11507936507936507%
Dev Error Preferred Model: 2.8095238095238098%
